# Multivariate Normal in Python

In [1]:
import numpy as np

## Generating random variables

### Univariate Normal

In [2]:
from numpy.random import default_rng

In [3]:
gen = default_rng(100) ## manually setting the seed

In [4]:
type(gen)

numpy.random._generator.Generator

In [5]:
gen = default_rng() # random seed
state = gen.bit_generator.state ## generating the state
state

{'bit_generator': 'PCG64',
 'state': {'state': 303569912328666541443949327325712330023,
  'inc': 283482735468249341487105010547600403213},
 'has_uint32': 0,
 'uinteger': 0}

We can save that state to a file and then use it again as follows

In [6]:
gen.bit_generator.state = state

In [7]:
x = gen.standard_normal((100_000, 5))

In [8]:
x.shape

(100000, 5)

In [9]:
x[:,0].mean(), x[:,0].std()

(-0.002625803489919437, 0.9987820113920508)

In [10]:
x.mean(axis=0)

array([-2.62580349e-03,  2.80629830e-04, -4.06825538e-05,  1.85665784e-03,
       -1.43839915e-03])

In [11]:
x.std(axis=0)

array([0.99878201, 1.00062503, 0.99844802, 1.00205125, 1.00131516])

In [12]:
x = gen.normal(1.5, 2, (100_000, 5))

In [13]:
x.mean(axis=0)

array([1.49842377, 1.50102508, 1.50276884, 1.49446717, 1.49841479])

In [14]:
x.std(axis=0)

array([1.99517824, 2.00016565, 1.99875554, 1.9995485 , 1.99662529])

### Multivariate Normal

In [15]:
mu = np.zeros((5))

In [16]:
# generate a random A, and compute Sigma as the outer product
A = gen.normal(1, 1, (5,5))
Sigma = A@A.T

In [17]:
Sigma

array([[ 7.16728024,  0.68257781,  8.56618681,  3.08237592,  4.65587799],
       [ 0.68257781,  8.98253061,  1.23792905, -6.24665935, -2.44137091],
       [ 8.56618681,  1.23792905, 13.18128218,  0.72701952,  4.38754789],
       [ 3.08237592, -6.24665935,  0.72701952, 12.25399676,  6.15596626],
       [ 4.65587799, -2.44137091,  4.38754789,  6.15596626,  4.65355445]])

In [18]:
z = gen.multivariate_normal(mean=mu,  cov=Sigma, size=1_000_000)

In [19]:
z.shape

(1000000, 5)

In [20]:
z.mean(axis=0)

array([-0.00101216,  0.0019855 , -0.00221562, -0.00039861, -0.00081751])

In [21]:
z.var(axis=0)

array([ 7.17771668,  8.97148621, 13.19093883, 12.24809059,  4.65681377])

In [22]:
Sigma.diagonal()

array([ 7.16728024,  8.98253061, 13.18128218, 12.25399676,  4.65355445])

In [23]:
np.diag(Sigma.diagonal())

array([[ 7.16728024,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  8.98253061,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , 13.18128218,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , 12.25399676,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  4.65355445]])

### Legacy approach to setting state

In [24]:
np.random.seed(42)

In [25]:
st = np.random.get_state()

In [26]:
np.random.set_state(st)

In [27]:
z2 = np.random.multivariate_normal(mean=mu, cov=Sigma, size=10000)

In [28]:
z2[0]

array([-0.07680725,  1.64047347, -2.14777747, -1.39090491, -0.44627057])

In [29]:
z2[0]

array([-0.07680725,  1.64047347, -2.14777747, -1.39090491, -0.44627057])

In [30]:
z2.mean(axis=0)

array([-0.02184029, -0.04967203, -0.02203345,  0.02557707,  0.00058157])

In [31]:
z2.var(axis=0)

array([ 7.34450447,  9.07011249, 13.51425697, 12.22751783,  4.73572826])

## Task 1
Set `gen = default_rng(100)`, and generate 10000 samples from bivariate normal with mean (10, 10) and variances 3 and 1 and covariance 1. Compute the sample covariance matrix __without__ using the `np.cov()` method. Compare the result with the obtained using `np.cov`.

In [32]:
gen = default_rng(100)
N = 10_000

mean = np.array([10,10])
cov = np.array([[3,1],[1,5]])

x = gen.multivariate_normal(mean=mean, cov=cov, size=N)

u = x - x.mean(0)
cov = u.T@u/(N-1)

print(cov)
print('*'*30)
print(np.cov(x, rowvar=False))
print('*'*30)
np.testing.assert_almost_equal(cov, np.cov(x, rowvar=False))

[[2.96065507 1.00828446]
 [1.00828446 5.03174172]]
******************************
[[2.96065507 1.00828446]
 [1.00828446 5.03174172]]
******************************


## Task 2
Let $z_1$ and $z_2$ be bivariate normal with means 0, variances 1, and covariance .9. Compute the conditional mean and variance of $z_1$, given that $z_2=1$.

$$
p(z_1 | z_2) = \mathcal{N}\left(\mu_1 + \frac{\rho \sigma_{1}}{\sigma_{2}} (z_2 - \mu_2), (1 - \rho^2)\sigma_1^2\right)
$$


In [33]:
rho=.9
sigma1 = 1
sigma2 = 1
mu1=0
mu2=0

z2 = 1

(
mu1 + (z2 - mu2)*(rho*sigma1)/(sigma2), 
(1 - rho**2)*sigma1**2
)

(0.9, 0.18999999999999995)

## General case

$$
\begin{align}
\operatorname{E}(\mathbf{z}_1 | \mathbf{z}_2) &= \boldsymbol \mu_1 + \mathbf{\Sigma}_{1 2} \mathbf{\Sigma}^{-1}_{2 2} (\mathbf{z}_2 - \boldsymbol \mu_2) \\
\operatorname{cov}(\mathbf{z}_1 | \mathbf{z}_2) &= \mathbf{\Sigma}_{1 1} - \mathbf{\Sigma}_{1 2}\mathbf{\Sigma}_{2 2}^{-1}\mathbf{\Sigma}_{2 1}
\end{align}
$$

## Task 3
Compute the conditional mean and variance of $z_1$, given that $z_2=1$ Using


In [34]:
Sigma12 = np.array([[0.1]])
Sigma22 = np.array([[1.0]])
Sigma21 = np.array([[0.1]])
Sigma11 = np.array([[1.0]])

mu1 = np.array([0])
mu2 = np.array([0])

In [35]:
z2 = np.array([1])

cond_mu = mu1 + (Sigma12 / np.linalg.inv(Sigma22)) * (z2 - mu2)
cond_mu

array([[0.1]])

In [36]:
cond_cov = Sigma11 - Sigma12 @ np.linalg.inv(Sigma22) @ Sigma21
cond_cov

array([[0.99]])

Instead of computing the inverse $\mathbf{\Sigma}^{-1}_{2 2}$, it is recommended to get $\mathbf{\Sigma}^{-1}_{2 2} (\mathbf{z}_2 - \boldsymbol \mu_2)$, by solving the matrix equation

$$\mathbf{\Sigma}^{-1}_{2 2} B = (\mathbf{z}_2 - \boldsymbol \mu_2) $$

using the `linalg.solve()` function. Similarly, for $\mathbf{\Sigma}^{-1}_{2 2} \mathbf{\Sigma}_{2 1}$

In [37]:
cond_mu = mu1 + Sigma12 @ np.linalg.solve(Sigma22, z2 - mu2)

cond_cov = Sigma11 - Sigma12 @ np.linalg.solve(Sigma22, Sigma21)

cond_mu, cond_cov

(array([0.1]), array([[0.99]]))

## Task 4
Compute the conditional mean and variance of $z_1$, given that $z_2=1$ Using


In [38]:
Sigma12 = np.array([[0.9]])
Sigma22 = np.array([[1.0]])
Sigma21 = np.array([[0.9]])

mu1 = np.array([0])
mu1 = np.array([0])

In [39]:
z2 = np.array([1])
cond_mu = mu1 + Sigma12 @ np.linalg.solve(Sigma22, z2 - mu2)

cond_cov = Sigma11 - Sigma12 @ np.linalg.solve(Sigma22, Sigma21)

cond_mu, cond_cov

(array([0.9]), array([[0.19]]))

## Task 5

Use `A = gen.normal(1, 1, (4,4))` to compute `Sigma = A@A.T` Let $z$ be 4D Normal with covariance `Sigma` and partition it into $z_1$ and $z_2$ both of which are 2D.
Compute the conditional variance of $z_1$ given $z_2$


In [40]:
A = gen.normal(1, 1, (4,4))
Sigma = A@A.T

indices_1 = [0, 1]
indices_2 = [2, 3]

z2indices = np.zeros(4, bool)
z1indices = z2indices.copy()

z2indices[indices_2] = 1
z1indices[indices_1] = 1

Sigma11 = Sigma[z1indices, :][:, z1indices]
Sigma12 = Sigma[z1indices, :][:, z2indices]
Sigma21 = Sigma[z2indices, :][:, z1indices]
Sigma22 = Sigma[z2indices, :][:, z2indices]

cond_cov = Sigma11 - Sigma12 @ np.linalg.solve(Sigma22, Sigma21)
cond_cov

array([[ 3.95231817, -1.35320667],
       [-1.35320667,  0.62295154]])